<a href="https://colab.research.google.com/github/kinzafaiyaz/Predicting-NHL-player-Salaries/blob/main/Predict_Stock_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the libraries which will be use to predict the stock prices

In [65]:
####Importing Libraries
import sys
import pandas as pd
import numpy as np

In [66]:
# Check if local computer has the library yfinance. If not, install. Then Import it.
!{sys.executable} -m pip install yfinance # Check if the machine has yfinance, if not, download yfinance
import yfinance as yf # Import library to access Yahoo finance stock data

In [67]:
# Adds financial information and indicators 
!{sys.executable} -m pip install ta # Download ta
from ta import add_all_ta_features # Library that does financial technical analysis 

In [68]:
# Install fastai to use the date function
!{sys.executable} -m pip install fastai # Download fastai 
import fastai.tabular # Library that does date factors

In [69]:
!{sys.executable} -m pip install plotly # Download plotly 
import plotly.graph_objs as go  # Import the graph ojbects 

In [70]:
# Import sklearn modules that will help with modeling building

!{sys.executable} -m pip install sklearn # Download sklearn 
from sklearn.metrics import mean_squared_error # Install error metrics 
from sklearn.linear_model import LinearRegression # Install linear regression model
from sklearn.neural_network import MLPRegressor # Install ANN model 
from sklearn.preprocessing import StandardScaler # to scale for ann
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score

User provides the ticker of the stock

In [114]:
# Choose your ticker
tickerSymbol = "AMZN"

# Choose date range - format should be 'YYYY-MM-DD' 
startDate = '2017-01-01' # as strings
endDate = '2021-02-25' # as strings

Using yfinance package to get the data of the ticker symbol and previewing the data

---



In [112]:
# Create ticker yfinance object
tickerData = yf.Ticker(tickerSymbol)

# Create historic data dataframe and fetch the data for the dates given. 
df = tickerData.history(start = startDate, end = endDate)

print('-----------------------')
print('Done!')
print(df.head())

-----------------------
Done!
                  Open        High  ...  Dividends  Stock Splits
Date                                ...                         
2017-01-03  116.029999  117.839996  ...          0             0
2017-01-04  117.550003  119.660004  ...          0             0
2017-01-05  118.860001  120.949997  ...          0             0
2017-01-06  120.980003  123.879997  ...          0             0
2017-01-09  123.550003  125.430000  ...          0             0

[5 rows x 7 columns]


Changing the date/time from string to a data type

In [95]:
# Change the date column to a pandas date time column 

# Define string format
date_change = '%Y-%m-%d'

# Create a new date column from the index
df['Date'] = df.index

# Perform the date type change
df['Date'] = pd.to_datetime(df['Date'], format = date_change)

# Create a variable that is the date column
Dates = df['Date']

Getting more variables from ta_features package besides the open, high, low, close, volume,

In [96]:
# Add all technical analysis to the dataframe we've already loaded
df = add_all_ta_features(df, "Open", "High", "Low", "Close", "Volume", fillna=True) 

print(df.columns)

/usr/local/lib/python3.6/dist-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Date', 'volume_adi', 'volume_obv', 'volume_cmf', 'volume_fi',
       'volume_mfi', 'volume_em', 'volume_sma_em', 'volume_vpt', 'volume_nvi',
       'volume_vwap', 'volatility_atr', 'volatility_bbm', 'volatility_bbh',
       'volatility_bbl', 'volatility_bbw', 'volatility_bbp', 'volatility_bbhi',
       'volatility_bbli', 'volatility_kcc', 'volatility_kch', 'volatility_kcl',
       'volatility_kcw', 'volatility_kcp', 'volatility_kchi',
       'volatility_kcli', 'volatility_dcl', 'volatility_dch', 'volatility_dcm',
       'volatility_dcw', 'volatility_dcp', 'volatility_ui', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_sma_fast',
       'trend_sma_slow', 'trend_ema_fast', 'trend_ema_slow', 'trend_adx',
       'trend_adx_pos', 'trend_adx_neg', 'trend_vortex_ind_pos',
       'trend_vortex_ind_neg', 'trend_vortex_ind_diff', 'trend_trix',
       'trend_mass_index', 'trend_cci', 'trend_

Using add_datepart from Fastai tabular to add columns relevent to a data in the column and add_cyclic adds trigonometric date/time features to a date in the column

In [97]:
# Define the date parts 
fastai.tabular.add_datepart(df,'Date', drop = 'True')

# Ensure the correct format
df['Date'] = pd.to_datetime(df.index.values, format = date_change)

# Add the date parts
fastai.tabular.add_cyclic_datepart(df, 'Date', drop = 'True')

/usr/local/lib/python3.6/dist-packages/fastai/tabular/transform.py:63: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,Open,High,Low,Close,Volume,Dividends,Stock Splits,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,...,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,weekday_cos,weekday_sin,day_month_cos,day_month_sin,month_year_cos,month_year_sin,day_year_cos,day_year_sin
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-03,116.029999,117.839996,115.510002,116.860001,20663900,0,0,3.281424e+06,20663900,0.158800,0.000000e+00,50.000000,0.000000,0.000000,-7.861174e+06,1000.000000,116.736666,0.000000,116.860001,116.860001,116.860001,0.000000,0.000000,0.0,0.0,116.736666,119.066661,114.406672,3.991881,0.526467,0.0,0.0,115.510002,117.839996,116.674999,1.993834,0.579400,0.000000,0.000000,0.000000,...,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000,57.939993,57.939993,-42.060007,0.000000,116.860001,0.000000,0.000000,0.000000,0.000000,-37.968645,0.000000,0.000000,2017,1,1,3,1,3,False,False,False,False,False,False,1483401600,0.623490,0.781831,9.189578e-01,0.394356,1.000000,0.0,0.999407,0.034422
2017-01-04,117.550003,119.660004,117.290001,118.690002,19630900,0,0,6.843147e+06,40294800,0.169827,3.592458e+07,100.000000,21.731108,21.731108,-7.538387e+06,1015.659779,117.618467,0.000000,117.775002,119.605003,115.945000,3.107624,0.750000,0.0,0.0,117.641668,119.991666,115.291669,3.995180,0.723050,0.0,0.0,115.510002,119.660004,117.585003,3.523669,0.766265,0.000000,0.145983,0.029197,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,35.672569,76.626485,67.283239,-23.373515,0.000000,117.607802,0.000000,-0.400267,-0.080053,-0.320214,1.565978,1.553843,1.565978,2017,1,1,4,2,4,False,False,False,False,False,False,1483488000,-0.222521,0.974928,8.207634e-01,0.571268,1.000000,0.0,0.998667,0.051620
2017-01-05,118.860001,120.949997,118.320000,120.669998,19492200,0,0,2.218493e+07,59787000,0.371066,3.630600e+07,100.000000,15.651319,18.691214,6.325859e+05,1032.603094,118.388391,0.000000,118.740000,121.851654,115.628347,5.241121,0.810124,0.0,0.0,118.421111,120.864443,115.977780,4.126513,0.960209,0.0,0.0,115.510002,120.949997,118.230000,4.581434,0.948530,0.000000,0.416642,0.106686,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,53.866021,94.852959,76.473146,-5.147041,0.000000,118.795570,0.000000,-0.766068,-0.217256,-0.548811,1.668208,1.654446,3.260309,2017,1,1,5,3,5,False,False,False,False,False,False,1483574400,-0.900969,0.433884,6.889669e-01,0.724793,1.000000,0.0,0.997630,0.068802
2017-01-06,120.980003,123.879997,120.029999,123.410004,28545300,0,0,4.376082e+07,88332300,0.495411,4.229289e+07,100.000000,31.290599,22.891009,9.733371e+05,1032.603094,119.697701,0.000000,119.907501,124.767387,115.047616,8.106058,0.860348,0.0,0.0,119.425833,122.220832,116.630835,4.680726,1.212732,1.0,0.0,115.510002,123.879997,119.695000,6.980377,0.943848,0.000000,0.842525,0.253853,...,0.000000,100.000000,0.000000,0.000000,0.000000,100.000000,65.116363,94.384781,88.621408,-5.615219,0.000000,120.469807,0.000000,2.409125,0.308020,2.101105,2.270660,2.245264,5.605000,2017,1,1,6,4,6,False,False,False,False,False,False,1483660800,-0.900969,-0.433884,5.289640e-01,0.848644,1.000000,0.0,0.996298,0.085965
2017-01-09,123.550003,125.430000,123.040001,124.900002,228804

Defining how far out I want to predict
For now, interested in 1 day later, 5 days, and 10 days later.

Using 75% of all the data fo training the model and the rest will be used for testing the model

In [98]:
# Define key model parameters

# Set days out to predict 
shifts = [1,5,10]

# Set a training percentage
train_pct = .75

# Plotting dimensions
w = 16 # width
h = 4 # height 

In order to do the days in the future,I have to move the closing costs by that number of days.

In [99]:
# Create the lags 
def CreateLags(df,lag_size):
  # inputs: dataframe , size of the lag (int)
  # ouptut: dataframe ( with extra lag column), shift size (int)

  # add lag
  shiftdays = lag_size
  shift = -shiftdays
  df['Close_lag'] = df['Close'].shift(shift)
  return df, shift
  


Function to split the data for training and testing

In [100]:
# Split the testing and training data 
def SplitData(df, train_pct, shift):
  # inputs: dataframe , training_pct (float between 0 and 1), size of the lag (int)
  # ouptut: x train dataframe, y train data frame, x test dataframe, y test dataframe, train data frame, test dataframe

  train_pt = int(len(df)*train_pct)
  
  train = df.iloc[:train_pt,:]
  test = df.iloc[train_pt:,:]
  
  x_train = train.iloc[:shift,1:-1]
  y_train = train['Close_lag'][:shift]
  x_test = test.iloc[:shift,1:-1]
  y_test = test['Close'][:shift]

  return x_train, y_train, x_test, y_test, train, test



Since we have a large number of features in the data. I am just defining a function here which goes through and makes sure the right columns are numbers (floats) and the right columns are categories (like strings)

In [101]:
# Ensure column types are correct

def CorrectColumnTypes(df):
  # Input: dataframe 
  # ouptut: dataframe (with column types changed)

  # Numbers
  for col in df.columns[1:80]:
      df[col] = df[col].astype('float')

  for col in df.columns[-10:]:
      df[col] = df[col].astype('float')

  # Categories 
  for col in df.columns[80:-10]:
      df[col] = df[col].astype('category')

  return df 

Function to plot our results

In [102]:
# Function to make the plots
def PlotModelResults_Plotly(train, test, pred, ticker, w, h, shift_days,name):
  # inputs: train dataframe, test dataframe, predicted value (list), ticker ('string'), width (int), height (int), shift size (int), name (string)
  # output: None

  # Create lines of the training actual, testing actual, prediction 
  D1 = go.Scatter(x=train.index,y=train['Close'],name = 'Train Actual') # Training actuals
  D2 = go.Scatter(x=test.index[:shift],y=test['Close'],name = 'Test Actual') # Testing actuals
  D3 = go.Scatter(x=test.index[:shift],y=pred,name = 'Our Prediction') # Testing predction

  # Combine in an object  
  line = {'data': [D1,D2,D3],
          'layout': {
              'xaxis' :{'title': 'Date'},
              'yaxis' :{'title': '$'},
              'title' : name + ' - ' + tickerSymbol + ' - ' + str(shift_days)
          }}
  # Send object to a figure 
  fig = go.Figure(line)

  # Show figure
  fig.show()
  

Defining a function which will be do simple linear regression

In [115]:
# Regreesion Function

def LinearRegression_fnc(x_train,y_train, x_test, y_test):
  #inputs: x train data, y train data, x test data, y test data (all dataframe's)
  # output: the predicted values for the test data (list)
  
  lr = LinearRegression()
  lr.fit(x_train,y_train)
  lr_pred = lr.predict(x_test)
  lr_MSE = mean_squared_error(y_test, lr_pred)
  lr_R2 = lr.score(x_test, y_test)
  
  print('Linear Regression R2: {}'.format(lr_R2))
  print('Linear Regression MSE: {}'.format(lr_MSE))
  

  return lr_pred


Defining a function which will do Lasso Regression 

In [116]:
###Lasso

def LinearLasso_func(x_train, y_train, x_test, y_test):
   #inputs: Training and Testing Data Set
   #Output: Predicted Values

   llr = linear_model.Lasso(alpha= 1.0)
   llr.fit(x_train, y_train)
   llr_pred = llr.predict(x_test)
   llr_MSE = mean_squared_error(y_test, llr_pred)
   llr_R2 = llr.score(x_test, y_test)
   
   print('Lasso Linear Regression R2: {}'.format(llr_R2))
   print('Lasso Linear Regression MSE: {}'.format(llr_MSE))
   
   return llr_pred



Defining a function for Artifical Neural Networks. 

In [117]:
# ANN Function 

def ANN_func(x_train,y_train, x_test, y_test):

  # Scaling data
  scaler = StandardScaler()
  scaler.fit(x_train)
  x_train_scaled = scaler.transform(x_train)
  x_test_scaled = scaler.transform(x_test)


  MLP = MLPRegressor(random_state=1, max_iter=1000, hidden_layer_sizes = (100,), activation = 'identity',learning_rate = 'adaptive').fit(x_train_scaled, y_train)
  MLP_pred = MLP.predict(x_test_scaled)
  MLP_MSE = mean_squared_error(y_test, MLP_pred)
  MLP_R2 = MLP.score(x_test_scaled, y_test)
  
  print('Muli-layer Perceptron R2 Test: {}'.format(MLP_R2))
  print('Multi-layer Perceptron MSE: {}'.format(MLP_MSE))
  

  return MLP_pred

Defining a function which show how much money I would have made, had I been trading this strategy

In [118]:
def CalcProfit(test_df,pred,j):
  pd.set_option('mode.chained_assignment', None)
  test_df['pred'] = np.nan
  test_df['pred'].iloc[:-j] = pred
  ##how much money gained or lost in one day
  test_df['change'] = test_df['Close_lag'] - test_df['Close'] 
  ##The difference between the prediction and actually closing price
  test_df['change_pred'] = test_df['pred'] - test_df['Close'] 
  ##If the function between is greater than 0, it means that we got the direction right (i.e. made profit)
  test_df['MadeMoney'] = np.where(test_df['change_pred']/test_df['change'] > 0, 1, -1) 
  ##Calculating how much money made or lost
  test_df['profit'] = np.abs(test['change']) * test_df['MadeMoney']
  profit_dollars = test['profit'].sum()
  print('Would have made: $ ' + str(round(profit_dollars,1)))
  profit_days = len(test_df[test_df['MadeMoney'] == 1])
  print('Percentage of good trading days: ' + str( round(profit_days/(len(test_df)-j),2))     )

  return test_df, profit_dollars

Generates results now!! :) 

In [119]:
# Go through each shift....

for j in shifts: 
  print(str(j) + ' days out:')
  print('------------')
  df_lag, shift = CreateLags(df,j)
  df_lag = CorrectColumnTypes(df_lag)
  x_train, y_train, x_test, y_test, train, test = SplitData(df, train_pct, shift)

  # Linear Regression
  print("Linear Regression")
  lr_pred = LinearRegression_fnc(x_train, y_train, x_test, y_test)
  test2, profit_dollars = CalcProfit(test,lr_pred,j)
  PlotModelResults_Plotly(train, test, lr_pred, tickerSymbol, w, h, j, 'Linear Regression')

  # Lasso Linear Regression
  print("Lasso Linear Regression")
  llr_pred = LinearLasso_func(x_train,y_train, x_test, y_test)
  test2, profit_dollars = CalcProfit(test,llr_pred,j)
  PlotModelResults_Plotly(train, test, llr_pred, tickerSymbol, w, h, j, 'Lasso Linear Regression')

  # Artificial Neuarl Network 
  print("ANN")
  MLP_pred = ANN_func(x_train,y_train, x_test, y_test)
  test2, profit_dollars = CalcProfit(test,MLP_pred,j)
  PlotModelResults_Plotly(train, test, MLP_pred, tickerSymbol, w, h, j, 'ANN')
  print('------------')



1 days out:
------------
Linear Regression
Linear Regression R2: 0.9991814505687867
Linear Regression MSE: 1.2317406921887382
Would have made: $ 23.1
Percentage of good trading days: 0.47


Lasso Linear Regression
Lasso Linear Regression R2: 0.9989371985369138
Lasso Linear Regression MSE: 1.5992874222155962
Would have made: $ 31.3
Percentage of good trading days: 0.47


ANN
Muli-layer Perceptron R2 Test: 0.9979638002861173
Multi-layer Perceptron MSE: 3.0640422549617163
Would have made: $ 58.1
Percentage of good trading days: 0.51


------------
5 days out:
------------
Linear Regression
Linear Regression R2: 0.9904099619450313
Linear Regression MSE: 14.519730940206731
Would have made: $ -78.1
Percentage of good trading days: 0.5


Lasso Linear Regression
Lasso Linear Regression R2: 0.9877817257154954
Lasso Linear Regression MSE: 18.498993867155463
Would have made: $ -100.6
Percentage of good trading days: 0.48


ANN
Muli-layer Perceptron R2 Test: 0.9871471261727335
Multi-layer Perceptron MSE: 19.459804925763052
Would have made: $ -92.6
Percentage of good trading days: 0.48


------------
10 days out:
------------
Linear Regression
Linear Regression R2: 0.9761857877069599
Linear Regression MSE: 36.4233772877413
Would have made: $ 95.3
Percentage of good trading days: 0.5


Lasso Linear Regression
Lasso Linear Regression R2: 0.972082391400361
Lasso Linear Regression MSE: 42.69944260525994
Would have made: $ -13.7
Percentage of good trading days: 0.49


ANN
Muli-layer Perceptron R2 Test: 0.9702384387770119
Multi-layer Perceptron MSE: 45.519732492429135
Would have made: $ 14.4
Percentage of good trading days: 0.48


------------
